In [ ]:
%pip install --upgrade pip

In [ ]:

%pip install scipy

In [ ]:
%pip install --default-timeout=1000 future --use-pep517

In [ ]:
%pip install tensorflow --default-timeout=10000
# import tensorflow as tf
# print(tf.__version__)

In [ ]:
import os
os.sys.path
# add open cv to path
os.sys.path.append('/usr/local/lib/python3.10/dist-packages')    
os.sys.path


In [1]:
import numpy as np
import cv2
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D
from keras.optimizers import Adam
from keras.layers import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
#from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

#from keras.utils import plot_model

from keras.utils.vis_utils import plot_model
#from livelossplot import PlotLossesKerasTF
from keras.utils import plot_model
import scipy
import os

In [2]:
train_dir = 'train'
val_dir = 'test'
img_size=48
batch_size=64
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(img_size,img_size),
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical')
val_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(img_size,img_size),
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical')

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [3]:
for i in os.listdir("train/"):
    print(str(len(os.listdir("train/"+i))) +" "+ i +" images")

3995 angry images
436 disgust images
4097 fear images
7215 happy images
4965 neutral images
4830 sad images
3171 surprise images


In [4]:
for i in os.listdir("test/"):
    print(str(len(os.listdir("test/"+i))) +" "+ i +" images")

958 angry images
111 disgust images
1024 fear images
1774 happy images
1233 neutral images
1247 sad images
831 surprise images


In [2]:
emotion_model = Sequential()
emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))#output=(48-3+0)/1+1=46
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))#output=(46-3+0)/1+1=44
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))#output=devided input by 2 it means 22,22,64
emotion_model.add(Dropout(0.25))#reduce 25% module at a time of output
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu',input_shape=(48,48,1)))#(22-3+0)/1+1=20
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))#10
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))#(10-3+0)/1+1=8
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))#output=4
emotion_model.add(Dropout(0.25))#nothing change
emotion_model.add(Flatten())#here we get multidimention output and pass as linear to the dense so that 4*4*128=2048
emotion_model.add(Dense(1024, activation='relu'))#hddien of 1024 neurons of input 
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))#hddien of 7 neurons of input
plot_model(emotion_model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)#save model leyer as model_plot.png
emotion_model.summary()

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 44, 44, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 22, 22, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 22, 22, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 20, 20, 128)       73856     
                                       

In [4]:
emotion_model.compile(loss='categorical_crossentropy',optimizer=Adam(learning_rate=0.0001, decay=1e-6),metrics=['accuracy'])
emotion_model_info = emotion_model.fit(
        train_generator,
        steps_per_epoch=448,
        epochs=20,
        validation_data=validation_generator,
        validation_steps=112)

Epoch 1/20
448/448 [==============================] - 1700s 4s/step - loss: 1.8068 - accuracy: 0.2556 - val_loss: 1.7296 - val_accuracy: 0.3333
Epoch 2/20
448/448 [==============================] - 196s 437ms/step - loss: 1.6449 - accuracy: 0.3575 - val_loss: 1.5543 - val_accuracy: 0.3996
Epoch 3/20
448/448 [==============================] - 198s 441ms/step - loss: 1.5389 - accuracy: 0.4101 - val_loss: 1.4653 - val_accuracy: 0.4435
Epoch 4/20
448/448 [==============================] - 199s 444ms/step - loss: 1.4682 - accuracy: 0.4345 - val_loss: 1.4016 - val_accuracy: 0.4654
Epoch 5/20
448/448 [==============================] - 243s 542ms/step - loss: 1.4060 - accuracy: 0.4644 - val_loss: 1.3534 - val_accuracy: 0.4867
Epoch 6/20
448/448 [==============================] - 213s 476ms/step - loss: 1.3556 - accuracy: 0.4863 - val_loss: 1.3106 - val_accuracy: 0.5000
Epoch 7/20
448/448 [==============================] - 193s 432ms/step - loss: 1.3139 - accuracy: 0.5038 - val_loss: 1.2953 - v

In [5]:
emotion_model.save_weights('new_model20.h5')

In [4]:
# load emotion model
emotion_model.load_weights('model50.h5')

In [9]:
# start the webcam feed
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Natural", 5: "Sad", 6: "Surprised"}

#cap = cv2.VideoCapture('facial_exp.mkv')
cap = cv2.VideoCapture(0)
while True:
    # Find haar cascade to draw bounding box around face
    ret, frame = cap.read()
    #frame = cv2.flip(frame, 1)
    if not ret:
        break
    bounding_box = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    num_faces = bounding_box.detectMultiScale(gray_frame,scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)
        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow('Video', cv2.resize(frame,(600,500),interpolation = cv2.INTER_CUBIC))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 31ms/step


KeyboardInterrupt: 

In [6]:
import pickle
with open('logmodel50.pkl', 'wb') as fid:
    pickle.dump(emotion_model, fid,2) 

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\conv2d
......vars
.........0
.........1
...layers\conv2d_1
......vars
.........0
.........1
...layers\conv2d_2
......vars
.........0
.........1
...layers\conv2d_3
......vars
.........0
.........1
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\dropout
......vars
...layers\dropout_1
......vars
...layers\dropout_2
......vars
...layers\flatten
......vars
...layers\max_pooling2d
......vars
...layers\max_pooling2d_1
......vars
...layers\max_pooling2d_2
......vars
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2022-11-30 21:17:48         4662
metadata.json                                  2022-11-30 21:17:48           64
variables.h5                                   2022-11-30 21:17:48      9414592


In [8]:
model_json=emotion_model.to_json()
with open("new_model15.json","w")as json_file:
    json_file.write(model_json)